In [1]:
import os

from datasets import load_dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer

from peft import get_peft_model, TaskType

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

os.environ["WANDB_PROJECT"] = "Llama-3.2-1B-Instruct-PPO"
os.environ["WANDB_ENTITY"] = "RADFAN"

In [2]:
# =============================================================================
# Configs
# =============================================================================

# Model config
# =============================================================================

model_config = ModelConfig(
    model_name_or_path   = "meta-llama/Llama-3.2-1B-Instruct",
    # LoRA
    # -------------------------------------------------------------------------
    use_peft             = True,
    lora_task_type       = TaskType.SEQ_CLS,
    lora_r               = 8,
    lora_alpha           = 8,
    lora_dropout         = 0.05,
    lora_target_modules  = None,
    lora_modules_to_save = None,
    # Quantization
    # -------------------------------------------------------------------------
    load_in_8bit         = False,
    load_in_4bit         = False,
)

# Reward trainer config
# =============================================================================

training_args = RewardConfig(
    # Reward trainer params
    # -------------------------------------------------------------------------
    max_length                  = 2048,
    dataset_num_proc            = 8,
    center_rewards_coefficient  = None,
    # Common
    # -------------------------------------------------------------------------
    output_dir                  = f"{model_config.model_name_or_path}-reward",
    run_name                    = f"{model_config.model_name_or_path}-{model_config.lora_r}",
    per_device_train_batch_size = 8,
    num_train_epochs            = 0.1,
    gradient_checkpointing      = False,
    gradient_accumulation_steps = 1,
    learning_rate               = 1e-4,

    # Frequency of logs
    # -------------------------------------------------------------------------
    logging_steps               = 25,

    # Evaluation
    # -------------------------------------------------------------------------
    eval_strategy               = "steps",
    eval_steps                  = 125,

    # Push to hub after training
    # -------------------------------------------------------------------------
    push_to_hub                 = True,
    hub_model_id                = "RLHF-And-Friends/Llama-3.2-1B-Instruct-Reward"
)

# Script config
# =============================================================================

script_args = ScriptArguments(
    dataset_name        = "trl-lib/ultrafeedback_binarized",
    dataset_train_split = "train",
    dataset_test_split  = "test",
)

In [3]:
# =============================================================================
# Model & Tokenizer
# =============================================================================

# Tokenizer
# -----------------------------------------------------------------------------

tokenizer = AutoTokenizer.from_pretrained(
    model_config.model_name_or_path, use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token

# Model
# -----------------------------------------------------------------------------

quantization_config = get_quantization_config(model_config)
lora_config = get_peft_config(model_config)

model = AutoModelForSequenceClassification.from_pretrained(
    model_config.model_name_or_path, 
    num_labels = 1,
    quantization_config = quantization_config,
    device_map = (
        get_kbit_device_map() if quantization_config is not None 
        else "auto"
    ),
    use_cache = False if training_args.gradient_checkpointing else True,
)

model = get_peft_model(model, lora_config)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
     

In [5]:
# =============================================================================
# Load dataset
# =============================================================================

dataset = load_dataset(script_args.dataset_name)

train_dataset = dataset[script_args.dataset_train_split]
eval_dataset = (
    dataset[script_args.dataset_test_split] 
    if training_args.eval_strategy != "no" 
    else None
)

In [6]:
# =============================================================================
# Training
# =============================================================================

trainer = RewardTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: evgurovv (RADFAN). Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 344.00 MiB. GPU 0 has a total capacity of 15.63 GiB of which 58.56 MiB is free. Process 3039221 has 2.10 GiB memory in use. Including non-PyTorch memory, this process has 13.46 GiB memory in use. Of the allocated memory 13.15 GiB is allocated by PyTorch, and 119.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
# =============================================================================
# Save model and push to Hub
# =============================================================================

trainer.save_model(training_args.output_dir)
if training_args.push_to_hub:
    trainer.push_to_hub(dataset_name=script_args.dataset_name)

/home/gurovev/dev/rlhf-and-friends/FedPPO/.fed-ppo-venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [8]:
# =============================================================================
# Evaluate Model
# =============================================================================

if training_args.eval_strategy != "no":
    metrics = trainer.evaluate()
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>user                              │ <|im_start|>user                             │ [0.4758, 0.5242] │
│ As an HR manager, you want to test a          │ As an HR manager, you want to test a         │                  │
│ potential employee's ability to solve puzzles │ potential employee's ability to solve        │                  │
│ to determine their suitability for a job.     │ puzzles to determine their suitability for a │                  │
│ Write a Python script that generates a list   │ job. Write a Python script that generates a  │                  │
│ of questions that require logical reasoning   │ list of questions that require logical       │                  │
│ to answer. Your list should include questions │ reasoning to answer. Your list should        │                  │
│ related to mathematical puzzles, language     │ include questions related to mathematical    │                  │
│ puzzles, logic puzzles, lateral thinking      │ puzzles, language puzzles, logic puzzles,    │                  │
│ puzzles, and pattern recognition puzzles. Use │ lateral thinking puzzles, and pattern        │                  │
│ the following code as a starting point:       │ recognition puzzles. Use the following code  │                  │
│ questions = {                                 │ as a starting point:                         │                  │
│     "Mathematical puzzles": ["If the value of │ questions = {                                │                  │
│ x+y = 20 and x-y = 10, what is the value of x │     "Mathematical puzzles": ["If the value   │                  │
│ and y?", "If a pizza has a radius of 8 inches │ of x+y = 20 and x-y = 10, what is the value  │                  │
│ and is cut into 6 equal slices, what is the   │ of x and y?", "If a pizza has a radius of 8  │                  │
│ area of each slice?"],                        │ inches and is cut into 6 equal slices, what  │                  │
│     "Language puzzles": ["What word starts    │ is the area of each slice?"],                │                  │
│ with 'e' and ends with 'e' but only contains  │     "Language puzzles": ["What word starts   │                  │
│ one letter?", "I am taken from a mine, and    │ with 'e' and ends with 'e' but only contains │                  │
│ shut up in a wooden case, from which I am     │ one letter?", "I am taken from a mine, and   │                  │
│ never released, and yet I am used by almost   │ shut up in a wooden case, from which I am    │                  │
│ every person. What am I?"],                   │ never released, and yet I am used by almost  │                  │
│     "Logic puzzles": ["You have 3 boxes. One  │ every person. What am I?"],                  │                  │
│ contains only apples, one contains only       │     "Logic puzzles": ["You have 3 boxes. One │                  │
│ oranges, and one contains both apples and     │ contains only apples, one contains only      │                  │
│ oranges. The boxes have been incorrectly      │ oranges, and one contains both apples and    │                  │
│ labeled such that no label identifies the     │ oranges. The boxes have been incorrectly     │                  │
│ actual contents of the box it labels. Opening │ labeled such that no label identifies the    │                  │
│ just one box, and without looking in the box, │ actual contents of the box it labels.        │                  │
│ you take out one piece of fruit. From that    │ Opening just one box, and without looking in │                  │
│ one piece of fruit, how can you immediately   │ the bo

***** eval metrics *****
  epoch                   =     0.1001
  eval_accuracy           =     0.7718
  eval_loss               =     0.6931
  eval_runtime            = 0:00:18.67
  eval_samples_per_second =     53.484
  eval_steps_per_second   =      6.692


/home/gurovev/dev/rlhf-and-friends/FedPPO/.fed-ppo-venv/lib/python3.10/site-packages/trl/trainer/utils.py:763: UserWarning: There are 528 out of 999 instances where the predictions for both options are equal. As a consequence the accuracy can be misleading.
  warnings.warn(
